<h1>CS4619: Artificial Intelligence II</h1>
<h1>Deliberative Agents</h1>
<h2>
    Derek Bridge<br>
    School of Computer Science and Information Technology<br>
    University College Cork
</h2>

<h1>Initialization</h1>
$\newcommand{\Set}[1]{\{#1\}}$ 
$\newcommand{\Tuple}[1]{\langle#1\rangle}$ 
$\newcommand{\v}[1]{\pmb{#1}}$ 
$\newcommand{\cv}[1]{\begin{bmatrix}#1\end{bmatrix}}$ 
$\newcommand{\rv}[1]{[#1]}$ 
$\DeclareMathOperator{\argmax}{arg\,max}$ 
$\DeclareMathOperator{\argmin}{arg\,min}$ 
$\DeclareMathOperator{\dist}{dist}$
$\DeclareMathOperator{\abs}{abs}$

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

<h1>Thinking ahead</h1>
<ul>
    <li>We have been looking at agents that seek to maximize rewards. Now we'll look at agents that seek to achieve goals.</li>
    <li>But also, we have been looking at reactive agents: actions are chosen based on the current percept alone.</li>
    <li>In the sense-plan-act cycle, the plan phase should generally be more deliberative, especially for agents that seek to achieve goals.</li>
    <li>Thinking ahead is a form of simulation:
        <ul>
            <li>Trying out actions on a mental representation prior to executing the actions 
                in the actual world.
            </li>
        </ul>
    </li>
    <li>Class exercise: Give precise reasons why thinking ahead is advantageous: what can go 
        wrong if you don’t think ahead?
    </li>
    <li>Class exercise: Are there times when thinking ahead is disadvantageous: what can go 
        wrong if you do think ahead?
    </li>
</ul>

<h1>Planning sequences of actions</h1>
<ul>
    <li>Often, the agent will plan whole sequences of actions.</li>
    <li>But, there are at least two ways of integrating this with execution:
        <figure>
            <img src="images/planning1.png" /><br />
            <img src="images/planning2.png" />
        </figure>
    </li>
    <li>
        Class exercise: The second approach appears to be wasteful. But the first approach is suitable only for certain 
        environments. What kinds of environments?
    </li>
</ul>

<h1>State space</h1>
<ul>
    <li>We implement deliberation (thinking ahead) as a form of <b>search</b> through a directed graph.
    <li><b>State space</b>:
        <ul>
            <li>all states reachable by sequences of actions from some start state.</li>
        </ul>
    </li>
    <li>Represented by a directed graph in which: 
        <ul>
            <li>nodes represent states of the world;</li>
            <li>edges represent actions (state transformations).</li>
        </ul>
    </li>
    <li>The task is to find a <b>path</b> from the node labelled by the <b>start state</b> 
        to one of the nodes labelled by <b>goal states</b>.
    </li>
</ul>

<h1>Explicit state spaces</h1>
<ul>
    <li>The state space is just a directed graph: nodes and edges.</li>
    <li>So we can choose any data structure for representing graphs that you've encountered previously, e.g.:
        <ul>
            <li>adjacency list; or</li>
            <li>adjacency matrix.</li>
        </ul>
    </li>
    <li>This is fine if the number of states is small.</li>
    <li>This is the kind of representation that we would use for pathfinding &mdash; see later lecture.</li>
</ul>

<h1>Implicit state spaces</h1>
<ul>
    <li>But, in AI, the graph is often too large to specify and store explicitly: too many states.</li>
    <li>Instead, specify it implicitly by giving:
        <ul>
            <li>the start state;</li>
            <li>the set of operators for transforming states to other states; and</li>
            <li>the goal condition that can detect whether a state is a goal state.</li>
        </ul>
    </li>
    <li>There can also be a path cost function, $g$.</li>
</ul>

<h1>The 8-puzzle</h1>
<ul>
    <li>Sliding 8 numbered tiles around a $3 \times 3$ grid.</li>
    <li>How to represent the states:
        <ul>
            <li>$3 \times 3$ array of integers.</li>
        </ul>
    </li>
    <li>Start state, e.g.:
        <table>
            <tr><td style="border: 1px solid black;">2</td><td style="border: 1px solid black;">8</td><td style="border: 1px solid black;">3</td></tr>
            <tr><td style="border: 1px solid black;">1</td><td style="border: 1px solid black;">6</td><td style="border: 1px solid black;">4</td></tr>
            <tr><td style="border: 1px solid black;">7</td><td style="border: 1px solid black;"> </td><td style="border: 1px solid black;">5</td></tr>
        </table>
    </li>
    <li>Operators:
        <ul>
            <li>If blank is not leftmost, move it left by 1</li>
            <li>If blank is not uppermost, move it up by 1</li>
            <li>Etc.</li>
        </ul>
    </li>
    <li>Goal state, e.g.:
        <table>
            <tr><td style="border: 1px solid black;">1</td><td style="border: 1px solid black;">2</td><td style="border: 1px solid black;">3</td></tr>
            <tr><td style="border: 1px solid black;">8</td><td style="border: 1px solid black;"> </td><td style="border: 1px solid black;">4</td></tr>
            <tr><td style="border: 1px solid black;">7</td><td style="border: 1px solid black;">6</td><td style="border: 1px solid black;">5</td></tr>
        </table>
    </li>
</ul>

<h1>8-puzzle state space</h1>
<figure>
    <img src="images/8puzzle.png" />
</figure>
<ul>
    <li>This state space has $9!=362,800$ states.</li>
</ul>

<h1>The water jugs problem</h1>
<ul>
    <li>A 4-gallon jug and 3-gallon jug with no measuring markers, and a tap.</li>
    <li>Must get exactly 2 gallons into the 4-gallon jug.</li>
    <li>Representation of states:
        <ul>
            <li>Pair of integers, $\Tuple{x, y}$</li>
            <li>$x$ is the amount of water in the 4-gallon jug, $x \in \Set{0,1,2,3,4}$</li>
            <li>$y$ is the amount of water in the 3-gallon jug, $y \in \Set{0,1,2,3}$</li>
        </ul>
    </li>
    <li>Start state: $\Tuple{0,0}$</li>
    <li>Goal state: $\Tuple{2,n}$</li>
</ul>

<h1>The water jugs operators</h1>
<ol>
    <li>If $x < 4$ then $\Tuple{4, y}$</li>
    <li>If $y < 3$ then $\Tuple{x, 3}$</li>
    <li>If $x > 0$ then $\Tuple{0, y}$</li>
    <li>If $y > 0$ then $\Tuple{x, 0}$</li>
    <li>If $x+y \geq 4$ then $\Tuple{4, y-(4-x)}$</li>
    <li>If $x+y \geq 3$ then $\Tuple{x-(3-y),3}$</li>
    <li>If $x+y \leq 4 \land y > 0$ then $\Tuple{x+y, 0}$</li>
    <li>If $x+y \leq 3 \land x > 0$ then $\Tuple{0, x+y}$</li>
</ol>

<h1>The water jugs state space</h1>
<figure>
    <img src="images/jugs.png" />
</figure>

<h1>Applications of state space search</h1>
<ul>
    <li>Route planning</li>
    <li>Pathfinding in games</li>
    <li>Cargo loading</li>
    <li>Automatic assembly</li>
    <li>&hellip;</li>
</ul>